In [1]:
from quantopian.pipeline import Pipeline
from quantopian.research import run_pipeline
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.factors import SimpleMovingAverage, AverageDollarVolume

###Combining Filters
Like factors, filters can be combined. Combining filters is done using the `&` (and) and `|` (or) operators. For example, let's say we want to screen for securities that are in the top 10% of average dollar volume and have a latest close price above $20. To start, let's make a high dollar volume filter using an `AverageDollarVolume` factor and `percentile_between`:

In [2]:
dollar_volume = AverageDollarVolume(window_length=30)
high_dollar_volume = dollar_volume.percentile_between(90, 100)

Note: `percentile_between` is a `Factor` method returning a `Filter`.

Next, let's create a `latest_close` factor and define a filter for securities that closed above $20:

In [3]:
latest_close = USEquityPricing.close.latest
above_20 = latest_close > 20

Now we can combine our `high_dollar_volume` filter with our `above_20` filter using the `&` operator:

In [4]:
tradeable_filter = high_dollar_volume & above_20

This filter will evaluate to `True` for securities where both `high_dollar_volume` and `above_20` are `True`. Otherwise, it will evaluate to `False`. A similar computation can be made with the `|` (or) operator.

If we want to use this filter as a screen in our pipeline, we can simply pass `tradeable_filter` as the `screen` argument.

In [5]:
def make_pipeline():

    mean_close_10 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=10)
    mean_close_30 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=30)

    percent_difference = (mean_close_10 - mean_close_30) / mean_close_30

    dollar_volume = AverageDollarVolume(window_length=30)
    high_dollar_volume = dollar_volume.percentile_between(90, 100)

    latest_close = USEquityPricing.close.latest
    above_20 = latest_close > 20

    tradeable_filter = high_dollar_volume & above_20

    return Pipeline(
        columns={
            'percent_difference': percent_difference
        },
        screen=tradeable_filter
    )

When we run this, our pipeline output now only includes ~700 securities.

In [6]:
result = run_pipeline(make_pipeline(), '2019-05-05', '2019-05-05')
print 'Number of securities that passed the filter: %d' % len(result)
result

Number of securities that passed the filter: 794


percent_difference
2019-05-06 00:00:00+00:00 Equity(2 [HWM])                 0.045815
                          Equity(24 [AAPL])               0.036887
                          Equity(53 [ABMD])              -0.025557
                          Equity(62 [ABT])               -0.001032
                          Equity(67 [ADSK])               0.044820
                          Equity(76 [TAP])                0.023185
                          Equity(114 [ADBE])              0.030868
                          Equity(122 [ADI])               0.031932
                          Equity(128 [ADM])               0.002744
                          Equity(161 [AEP])               0.005391
                          Equity(185 [AFL])               0.003568
                          Equity(216 [HES])               0.023028
                          Equity(239 [AIG])               0.028965
                          Equity(266 [AJG])               0.028962
                          Equity(300 [ALK])               0.036741
                          Equity(337 [AMAT])              0.037732
                          Equity(351 [AMD])               0.017213
                          Equity(353 [AME])               0.026951
                          Equity(368 [AMGN])             -0.039630
                          Equity(438 [AON])               0.023199
                          Equity(448 [APA])              -0.030744
                          Equity(455 [APC])               0.244091
                          Equity(460 [APD])               0.036405
                          Equity(465 [APH])               0.007562
                          Equity(630 [ADP])               0.008956
                          Equity(679 [AXP])               0.034844
                          Equity(693 [AZO])              -0.000556
                          Equity(698 [BA])               -0.001155
                          Equity(700 [BAC])               0.037768
                          Equity(734 [BAX])              -0.026404
...                                                            ...
                          Equity(51653 [VXX])            -0.070034
                          Equity(51689 [PAGS])           -0.066805
                          Equity(51734 [VICI])            0.018849
                          Equity(51832 [ZS])             -0.007502
                          Equity(51854 [DBX])             0.060313
                          Equity(51887 [IQ])             -0.024257
                          Equity(51895 [SPOT])           -0.020109
                          Equity(51937 [DOCU])            0.030091
                          Equity(51961 [SMAR])            0.027014
                          Equity(51999 [HUYA])           -0.056357
                          Equity(52038 [CGC])             0.095809
                          Equity(52064 [EVRG])           -0.003526
                          Equity(52116 [XLC])             0.029001
                          Equity(52206 [PDD])             0.007483
                          Equity(52211 [TLRY])           -0.091874
                          Equity(52424 [ELAN])           -0.011739
                          Equity(52425 [FTCH])           -0.027262
                          Equity(52491 [GH])             -0.061701
                          Equity(52537 [STNE])           -0.216606
                          Equity(52592 [LIN])             0.001633
                          Equity(52747 [DELL])            0.057653
                          Equity(52968 [FOXA])            0.017403
                          Equity(52969 [FOX])             0.020950
                          Equity(52991 [DOW])             0.009031
                          Equity(52998 [LEVI])            0.001276
                          Equity(53023 [LYFT])           -0.037528
                          Equity(53056 [ALC])             0.011070
                          Equity(53063 [JMIA])            0.016815
                          Eq

In the next lesson, we'll look at masking factors and filters.